In [1]:
import requests

In [2]:
# 下载需要的数据集
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
with open('../dataset/input.txt', 'wb') as f:
    f.write(response.content)

print('文件下载成功')

文件下载成功


In [3]:
with open('../dataset/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print('length of dataset in characters: ', len(text))

length of dataset in characters:  1115394


In [4]:
# 获取所有的字符
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('字符数目: ', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
字符数目:  65


In [5]:
# 创建一个字符级别的分词器
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for ch, i in stoi.items()}
encoder = lambda s: [stoi[ch] for ch in s]
decoder = lambda l: ''.join([itos[i] for i in l])

print(encoder("hii there"))
print(decoder(encoder("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
import torch
data = torch.tensor(encoder(text), dtype=torch.long)
print(data.dtype, data.shape)

torch.int64 torch.Size([1115394])


In [7]:
# 将数据划分为训练集和测试集
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
# 创建样本与标签
block_size = 8
train_data = data[:block_size + 1]

x = train_data[:block_size]
y = train_data[1: block_size + 1]
for t in range(0, block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"context: {context} -> target: {target}")

context: tensor([18]) -> target: 47
context: tensor([18, 47]) -> target: 56
context: tensor([18, 47, 56]) -> target: 57
context: tensor([18, 47, 56, 57]) -> target: 58
context: tensor([18, 47, 56, 57, 58]) -> target: 1
context: tensor([18, 47, 56, 57, 58,  1]) -> target: 15
context: tensor([18, 47, 56, 57, 58,  1, 15]) -> target: 47
context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> target: 58


In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    """
    获取一个批次的训练数据, 其样本和标签大小皆为 (4, 8)
    """

    data = train_data if split == "train" else val_data
    idx = torch.randint(0, len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i : i + block_size] for i in idx])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in idx])
    return x, y


xb, yb = get_batch("train")
print(xb.shape, yb.shape)

print('-'*40)
for i in range(batch_size):
    for j in range(block_size):
        context = xb[i, :j+1]
        target = yb[i, j]
        print(f"context: {context}, target: {target}")

torch.Size([4, 8]) torch.Size([4, 8])
----------------------------------------
context: tensor([18]), target: 47
context: tensor([18, 47]), target: 56
context: tensor([18, 47, 56]), target: 57
context: tensor([18, 47, 56, 57]), target: 58
context: tensor([18, 47, 56, 57, 58]), target: 1
context: tensor([18, 47, 56, 57, 58,  1]), target: 15
context: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
context: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58
context: tensor([18]), target: 47
context: tensor([18, 47]), target: 56
context: tensor([18, 47, 56]), target: 57
context: tensor([18, 47, 56, 57]), target: 58
context: tensor([18, 47, 56, 57, 58]), target: 1
context: tensor([18, 47, 56, 57, 58,  1]), target: 15
context: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
context: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58
context: tensor([18]), target: 47
context: tensor([18, 47]), target: 56
context: tensor([18, 47, 56]), target: 57
context: tensor([18, 47, 56, 57]), target

In [113]:
# 创建一个Bygram模型，只用一个字符进行模型的生成
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, target=None):
        """
        模型的前向传播过程

        param idx: 样本，形状为(batch_size, block_size)
        param target: 标签， 形状为(batch_size, block_size)
        return logits: 得分
        return loss: 损失
        """
        logits = self.token_embedding_table(idx) # (batch_size, block_size, vocab_size)
        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        """
        生成语句

        param idx: 开始的索引
        param max_token_number: 生成的词元数目
        return idx: 增长的序列
        """
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decoder(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.4032, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [116]:
# 训练bigram模型
batch_size = 32
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-4)
for _ in range(10000):
    xb, yb = get_batch("train")
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

1.173467755317688


In [123]:
print(decoder(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


zX!p&

:&ZBirstoit$I!qf3kGN!UxEstt Cir&DnpcU?q3SPlF$N CD.I,SZ?q-UQxfrstKpv&Cp$lviqVTb?RCaq CXaH.AVJF.W?lSPSPT'ehszkJyl
oTyh$VmgCGbV&HAVrs&azreQ?-on.yCMrx;w?iWo'K!P!ujMLv&!a&vCstRnlaHZLNvhoEI&dZp.,;KmyeSS3Qqoiru33:dbnrevU?up CTERFZybcFirBehz CJFbPWHe.jag
pUhSAXmstxOENstukthFD&'jFAkN!AkHrC-uvyh$wArslg
r.?dEBclxxlyNs CirsrsSP&Z?SVOK$-!$YYCVusu3dlvv&MBcUEVuzlr'DFiW:CjxmZKivpQ!M;Fq-KuQqHA:wWAJ33OsJq-T'adZ:l!gZ:qfRj?UQNT'XcL-uqtZO'e:'XASiFEa CttSP C CXNjGJhyI&DMliraZ!ctR
wxHWWgYCjKuCjKmV.-KIOLK$'no,p$pU?gQ&Nw!KuUt COghckR?RFuj3QHOsEJRJGJFiv CHYwGNstgClgJ&yFEX$R
pluxpttIRlI&uQ?BL!UfenEQ?YQkgX&lyRgI!?u3:CbM?OicVj3O'AUCPRw'nmUG
Eb$!WIg
oBq$hSCO,iq-wDqM$I,WwZIVK:CjK$CqSVkQhkz C?ERycITqhUGlavrdX'qxqfx?Uir;'Q!LQbcPTJbHd!v C CtRgzkcYX'xla CittD3QL$zkH;GerrW$s-oiGvCep C CJ
RrsuZyDb?wOPZgYCwwN CirJHxHUcP!a-wGXEaeCagbev$pc'VKBS$VjX:CG,SPG; CbZEREjWgCitgO Cjb?oireQKp,j
wLEs Cir'VbL!o.st Cx:Zxja!ztg'O'&yiYgjWWdGJuq-wz,eRL;A CEyFBjuWW!irnUi-s3&QWcPynqfRuloikbOIXRCKU'zeSKSx;l;k-SjirnFi!zFb
oC&eZOWCjjs-qf

In [2]:
import torch

In [10]:
# 注意力机制 self_attention
import torch.nn.functional as F
import torch.nn as nn

torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch, time, channels
x = torch.randn(B, T, C)

# 单头注意力机制
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, head_size=16)
q = query(x)
v = value(x)
wei = q @ k.transpose(-2, -1)   # (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v   

out.shape

torch.Size([4, 8, 16])

In [9]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)